# This notebook runs a Monte Carlo propagation in ADAM

In [11]:
from adam import PropagationParams
from adam import OpmParams
from adam import ConfigManager
from adam import ProjectsClient
from adam import ResultsClient
from adam import AdamProcessingService
from adam import MonteCarloResults

This sets up authenticated access to the server. It needs to be done before pretty much everything you want to do with ADAM.

In [2]:
# ConfigManager loads the config set up via adamctl.
# See the README at https://github.com/B612-Asteroid-Institute/adam_home/blob/master/README.md
config_manager = ConfigManager()
config_manager.set_default_env('dev')
config = config_manager.get_config()
project = ProjectsClient().get_project_from_config(config)
aps = AdamProcessingService()

## Example Inputs

The PropagationParameters sent to the propagation API will operate on either Keplerian or Cartesian elements.

In [3]:
# Keplerian elements state vector
keplerian_elements = {
    'semi_major_axis_km': 448793612,
    'eccentricity': 0.1,
    'inclination_deg': 90,
    'ra_of_asc_node_deg': 91,
    'arg_of_pericenter_deg': 92,
    'true_anomaly_deg': 93,
    'gm': 132712440041.9394
}

# Uncertainties on elements
keplerian_sigma = {
    'semi_major_axis': 100,
    'eccentricity': 0.001,
    'inclination': 1,
    'ra_of_asc_node': 2,
    'arg_of_pericenter': 3,
    'true_anomaly': 4,
}

# Cartesian state vector
state_vec = [-150874809.2,
             -187234595.3,
             -73785026.7,
             14.64403935,
             -11.75744819,
             -5.583528281]

cartesian_sigma = {
    "x": 10,
    "y": 10,
    "z": 10,
    "x_dot": 1,
    "y_dot": 1,
    "z_dot": 1    
}


### Set Parameters

Commented parameters are optional. Uncomment to use.

In [4]:
propagation_params = PropagationParams({
    'start_time': '2001-01-02T01:13:46.62Z',  # propagation start time in ISO format
    'end_time': '2010-12-31T01:13:46.62Z',    # propagation end time in ISO format

    #'keplerianSigma': keplerian_sigma,
    'description': 'Jupyter Example',
    'cartesianSigma' : cartesian_sigma,       # comment out if using keplerian
    'monteCarloDraws': 10,
    'propagationType': 'MONTE_CARLO',
    'stopOnImpact': True,
    'step_size': 86400,
    'stopOnCloseApproach': False,
    'stopOnImpactAltitudeMeters': 500000,
    'closeApproachRadiusFromTargetMeters': 7000000000
})

opm_params = OpmParams({
    'epoch': '2001-01-02T01:13:46Z',
    # 'keplerian_elements': keplerian_elements,
    'state_vector': state_vec,                # comment out if using keplerian
})


### Submit and Run Propagation

In [5]:
batch_run = aps.execute_batch_propagation(
    project, 
    propagation_params, 
    opm_params, 
    object_id="MyTestObject10", # optional field can be used for searching job sets later
    user_defined_id="MyTestRun10" # optional field can be used for searching job sets later
)

### Get Status

In [6]:
print(batch_run.check_status())
batch_run.wait_for_complete(max_wait_sec = 500, print_waiting = True)
print()
print(batch_run.check_status())

CREATED
CREATED
..........RUNNING
......
COMPLETED


### Get Number of Runs

In [7]:
runs_count = batch_run.get_result_ephemeris_count()
print(f'# Runs: {runs_count}')

# Runs: 11


### Get Summary Statistics

In [13]:
stats = batch_run.get_summary()
print(stats)
print(stats.get_misses())

MonteCarloSummary(misses=10, close_approaches=2, impacts=1, pc=0.09090909090909091)
10


### Get Ephemeris for specific run

In [9]:
eph = batch_run.get_result_ephemeris(2)
print(eph)

                       Epoch             X             Y             Z  \
0    2001-01-02 01:13:46.620 -1.508748e+08 -2.011344e+08  6.781218e+06   
1    2001-01-03 01:13:46.620 -1.496813e+08 -2.021521e+08  6.660213e+06   
2    2001-01-04 01:13:46.620 -1.484784e+08 -2.031571e+08  6.538794e+06   
3    2001-01-05 01:13:46.620 -1.472664e+08 -2.041496e+08  6.416969e+06   
4    2001-01-06 01:13:46.620 -1.460452e+08 -2.051294e+08  6.294745e+06   
...                      ...           ...           ...           ...   
3647 2010-12-28 01:13:44.620  1.177291e+08 -1.322383e+08 -1.277293e+07   
3648 2010-12-29 01:13:44.620  1.187780e+08 -1.300741e+08 -1.279998e+07   
3649 2010-12-30 01:13:44.620  1.198054e+08 -1.278864e+08 -1.282472e+07   
3650 2010-12-31 01:13:44.620  1.208110e+08 -1.256753e+08 -1.284713e+07   
3651 2010-12-31 01:13:46.620  1.208110e+08 -1.256753e+08 -1.284713e+07   

             Vx         Vy        Vz  
0     13.759689 -11.850956 -1.398094  
1     13.867910 -11.705752 -1.402

### Get ending state vector

In [12]:
end_state_vectors = batch_run.get_final_positions(MonteCarloResults.PositionOrbitType.CLOSE_APPROACH)
print("Close State Vectors")
print(f'First close end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(MonteCarloResults.PositionOrbitType.IMPACT)
print("Impact State Vectors")
print(len(end_state_vectors))
print(f'First impact end state: {end_state_vectors[0]}')

end_state_vectors = batch_run.get_final_positions(MonteCarloResults.PositionOrbitType.MISS)
print("Miss State Vectors")
print(len(end_state_vectors))
print(f'First miss end state: {end_state_vectors[0]}')


Close State Vectors
First close end state: [numpy.datetime64('2004-10-15T01:13:43.586422'), -153016310.60684204, 4703606844.01722, -3827499475.602997]
Impact State Vectors
1
First impact end state: [numpy.datetime64('2010-12-30T20:56:02.361059'), -4020708.8821298536, -5561542.295034641, -467280.3576559379]
Miss State Vectors
10
First miss end state: [numpy.datetime64('2010-12-31T01:13:46.620000'), -184781982105.40283, -156909115082.23712, -58538558652.59232]
